# Ceramic pieces spectral data gathering
Saves the spectral data of each piece in a rgb image of size 1x224. The red channel stores the mean reflectance for each spectral band, while the green channel stores the mean standard deviation of the reflectance for each spectral band. All the data is stored in the 0-255 range. For float uses, it should be devided by 255.

## Define functions and imports

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from skimage import io
from skimage import filters
import glob
import spectral.io.envi as envi
import shutil
import numpy as np
import cv2
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
from matplotlib.lines import Line2D

In [4]:
def openHDR(hdrImage, mainImage, mask, window, paperMask, paperWindow, usePaperCalibration = True):
    img = envi.open(hdrImage)#, mainImage
    Darkref = envi.open("./Ceramica/BW_Reference/capture/DARKREF_Im_ceramica_2020-12-15_14-54-13.hdr",
                        "./Ceramica/BW_Reference/capture/DARKREF_Im_ceramica_2020-12-15_14-54-13.raw")
    Whiteref = envi.open("./Ceramica/BW_Reference/capture/WHITEREF_Im_ceramica_2020-12-15_14-54-13.hdr",
                         "./Ceramica/BW_Reference/capture/WHITEREF_Im_ceramica_2020-12-15_14-54-13.raw")

    Darkref = np.repeat(Darkref[:,:,:],10, axis=0)
    Whiteref = np.repeat(Whiteref[:,:,:],10, axis=0)
    imgArray = img[:,:,:]
    
    PaperDark = Darkref[np.nonzero(paperMask)]
    PaperWhite =Whiteref[np.nonzero(paperMask)]
    Paper = ((imgArray[np.nonzero(paperMask)] - PaperDark)/(PaperWhite - PaperDark ))
    Paper = Paper.reshape(-1, Paper.shape[-1])
    paperMean = np.array([np.mean(Paper[:,x]) for x in range(Paper.shape[1])])
    #paperStd = np.array([np.std(Paper[:,x]) for x in range(Paper.shape[1])])
    #pixelValues = np.empty((0,224))
    
    Image = imgArray[np.nonzero(mask)]
    Dark = Darkref[np.nonzero(mask)]
    White = Whiteref[np.nonzero(mask)]
    Reflectancia = ((Image - Dark)/(White - Dark ))
    Reflectancia = Reflectancia.reshape(-1, Reflectancia.shape[-1])
    histogramMean = None
    histogramStd = None
    if usePaperCalibration:
        histogramMean = ([np.mean(Reflectancia[:,x]/paperMean[x]) for x in range(Reflectancia.shape[1])],img.bands.centers)
        histogramStd = [np.std(Reflectancia[:,x]/paperMean[x]) for x in range(Reflectancia.shape[1])]
    else:
        histogramMean = ([np.mean(Reflectancia[:,x]) for x in range(Reflectancia.shape[1])],img.bands.centers)
        histogramStd = [np.std(Reflectancia[:,x]) for x in range(Reflectancia.shape[1])]
    return histogramMean, histogramStd, Reflectancia

In [5]:
def getMaskPaper(hsvImage,otsu):
    mask = hsvImage[:,:,2]>otsu*1.25
    mask = mask * hsvImage[:,:,0]>60
    mask[:,:110] = 0
    mask[:,475:] = 0
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_ERODE, kernel)
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_ERODE, kernel)
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_ERODE, kernel)
    return mask

In [6]:
def getMaskPieces(hsvImage,otsu):
    
    
    hsvImage[:,:,0] = hsvImage[:,:,0]<40
    hsvImage[:,:,2] = hsvImage[:,:,2]>otsu*0.2
    hsvImage[:,:,2] = hsvImage[:,:,2]<otsu*0.75
    
    mask = hsvImage[:,:,0]*hsvImage[:,:,2]*hsvImage[:,:,1]>40
    mask[:,:110] = 0
    mask[:,475:] = 0
    
    kernel = np.ones((7,7),np.uint8)
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_CLOSE, kernel)
    kernel = np.ones((5,5),np.uint8)
    
    mask = cv2.morphologyEx(np.float32(mask), cv2.MORPH_ERODE, kernel)
    flipMask = np.flipud(mask)
    num_labels, labels_im = cv2.connectedComponents(np.uint8(flipMask),connectivity=8)
    labels_im = np.flipud(labels_im)
    pieces=[]
    for i in range(1,num_labels):
        section = labels_im[:,:]==i
        pieces.append(section)
    return mask, pieces, labels_im
    

In [36]:
def openAndCrop(folderPath, counter, classValue, paperWindow=7, window = 5, usePaperCalibration = True, showImages=True, saveMasks=True, loadMasks=True):#, corners, paperTopLeft, paperBottomRight
    imagePath = glob.glob(folderPath+'*.png')[0].replace('\\','/')
    hdrImage = glob.glob(folderPath+'*capture/*.hdr')[0].replace('\\','/')
    image = io.imread(imagePath)
    generateImage = saveMasks or not loadMasks
    mask, masks, labels, paperMask = None, None, None, None
    
    bagName = folderPath.split('/')[-3]
    side = folderPath.split('/')[-2].split('_')[-1]
    
    if generateImage:
        hsvImage = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsvImage[:,:110,2] = 0
        hsvImage[:,475:,2] = 0
        otsu = filters.threshold_otsu(hsvImage[:,:,2])
        mask,masks, labels = getMaskPieces(hsvImage.copy(),otsu)
        inverseMask = np.ones(mask.shape)-mask
        paperMask = getMaskPaper(hsvImage.copy(),otsu)
        paperMask = paperMask * inverseMask
        
        
    if saveMasks:
        name = "./pieces_data/masks/"+bagName+"_"+side
        maskName = "./NN_Dataset/masks/"+bagName+"_"+side
        imageName = "./NN_Dataset/images/"+bagName+"_"+side
        plt.imsave(name+"_paperMask.png", paperMask, cmap="gray")
        plt.imsave(name+"_ceramicMask.png", mask, cmap="gray")
        plt.imsave(name+"_labeledMask.png", labels, cmap="gray",vmin=0, vmax=255)
        shutil.copyfile(hdrImage, imageName+".hdr")
        shutil.copyfile("."+hdrImage.split(".")[1]+".raw", imageName+".raw")
        plt.imsave(maskName+".png", inverseMask*255+mask*classValue, cmap="gray",vmin=0, vmax=255)#
        
        
        
        
    if loadMasks:
        name = "./pieces_data/masks/"+bagName+"_"+side
        paperMask = io.imread(name+"_paperMask.png")[:,:,0]
        mask = io.imread(name+"_ceramicMask.png")[:,:,0]
        labels = io.imread(name+"_labeledMask.png")[:,:,0]
        masks = [labels[:,:]==i for i in range(1,np.amax(labels)+1)]
        
        
    for piece in masks:
        name = bagName+"_"+str("{:02d}".format(counter))+"_"+side
        print(name)
        counter+=1
        histogram, std, values = openHDR(hdrImage, imagePath, mask, window, paperMask, paperWindow)#,paperMean,paperStd
        xValues = histogram[1]
        mean = np.array(histogram[0])
        std = np.array(std)
        pieceData = np.zeros((1,len(xValues),3))
        pieceData[0,:,0] = np.clip(mean,0,1)
        pieceData[0,:,1] = np.clip(std,0,1)
        plt.imsave("./pieces_data/spectrum/"+name+".png", pieceData, vmin=0, vmax=1)
        np.savetxt("./pieces_data/spectrum/"+name+"_std.txt", std)
        np.savetxt("./pieces_data/spectrum/"+name+"_mean.txt", mean)
        
    return counter

## Sample gathering

In [37]:
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_01/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_01/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_02/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_02/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_03/1_A/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_03/1_B/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_04/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_04/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_05/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g1/g1_05/1_B/",counter,0)




counter = 1
counter = openAndCrop("./Ceramica/g2/g2_01/1_A/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_01/1_B/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_02/1_A/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_02/1_B/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_03/1_A/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_03/1_B/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_04/1_A/",counter,1)
counter = openAndCrop("./Ceramica/g2/g2_04/2_A/",counter,1)
counter = openAndCrop("./Ceramica/g2/g2_04/3_A/",counter,1)
counter = 1
counter = openAndCrop("./Ceramica/g2/g2_04/1_B/",counter,1)
counter = openAndCrop("./Ceramica/g2/g2_04/2_B/",counter,1)
counter = openAndCrop("./Ceramica/g2/g2_04/3_B/",counter,1)




counter = 1
counter = openAndCrop("./Ceramica/g3/g3_01/1_A/",counter,2)
counter = 1
counter = openAndCrop("./Ceramica/g3/g3_01/1_B/",counter,2)




counter = 1
counter = openAndCrop("./Ceramica/g4/g4_01/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g4/g4_01/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g4/g4_02/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g4/g4_02/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g4/g4_03/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g4/g4_03/1_B/",counter,0)




counter = 1
counter = openAndCrop("./Ceramica/g5/g5_01/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_01/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_02/1_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_02/1_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_03/1_A/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_03/2_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_03/1_B/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_03/2_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_04/1_A/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_04/2_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_04/1_B/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_04/2_B/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_05/1_A/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_05/2_A/",counter,0)
counter = 1
counter = openAndCrop("./Ceramica/g5/g5_05/1_B/",counter,0)
counter = openAndCrop("./Ceramica/g5/g5_05/2_B/",counter,0)




counter = 1
counter = openAndCrop("./Ceramica/g6/g6_01/1_A/",counter,3)
counter = 1
counter = openAndCrop("./Ceramica/g6/g6_01/1_B/",counter,3)
counter = 1
counter = openAndCrop("./Ceramica/g6/g6_02/1_A/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_02/2_A/",counter,3)
counter = 1
counter = openAndCrop("./Ceramica/g6/g6_02/1_B/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_02/2_B/",counter,3)
counter = 1
counter = openAndCrop("./Ceramica/g6/g6_03/1_A/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_03/2_A/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_03/3_A/",counter,3)
counter = 1
counter = openAndCrop("./Ceramica/g6/g6_03/1_B/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_03/2_B/",counter,3)
counter = openAndCrop("./Ceramica/g6/g6_03/3_B/",counter,3)

g1_01_01_A
g1_01_02_A
g1_01_01_B
g1_01_02_B
g1_02_01_A
g1_02_02_A
g1_02_03_A
g1_02_04_A
g1_02_05_A
g1_02_06_A
g1_02_07_A
g1_02_08_A
g1_02_09_A
g1_02_10_A
g1_02_11_A
g1_02_12_A
g1_02_13_A
g1_02_14_A
g1_02_15_A
g1_02_16_A
g1_02_17_A
g1_02_18_A
g1_02_19_A
g1_02_20_A
g1_02_21_A
g1_02_22_A
g1_02_23_A
g1_02_24_A
g1_02_01_B
g1_02_02_B
g1_02_03_B
g1_02_04_B
g1_02_05_B
g1_02_06_B
g1_02_07_B
g1_02_08_B
g1_02_09_B
g1_02_10_B
g1_02_11_B
g1_02_12_B
g1_02_13_B
g1_02_14_B
g1_02_15_B
g1_02_16_B
g1_02_17_B
g1_02_18_B
g1_02_19_B
g1_02_20_B
g1_02_21_B
g1_02_22_B
g1_02_23_B
g1_02_24_B
g1_03_01_A
g1_03_01_B
g1_04_01_A
g1_04_02_A
g1_04_03_A
g1_04_04_A
g1_04_01_B
g1_04_02_B
g1_04_03_B
g1_04_04_B
g1_05_01_A
g1_05_02_A
g1_05_03_A
g1_05_01_B
g1_05_02_B
g1_05_03_B
g2_01_01_A
g2_01_02_A
g2_01_03_A
g2_01_04_A
g2_01_05_A
g2_01_06_A
g2_01_07_A
g2_01_08_A
g2_01_09_A
g2_01_10_A
g2_01_11_A
g2_01_12_A
g2_01_13_A
g2_01_14_A
g2_01_15_A
g2_01_16_A
g2_01_17_A
g2_01_18_A
g2_01_19_A
g2_01_20_A
g2_01_21_A
g2_01_22_A
g2_01_23_A